In [ ]:
#@title Copyright 2020 Google LLC. Double-click here for license information.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the data
try:
    df = pd.read_csv('recipes.csv')
except FileNotFoundError:
    print("Error: recipes.csv not found. Please make sure it's in the same directory.")
    exit()

# Process numerical features
num_features = ['rating_avg', 'rating_val', 'total_time']
df[num_features] = df[num_features].apply(pd.to_numeric, errors='coerce')
df = df.dropna(subset=num_features)
scaler = StandardScaler()
num_matrix = scaler.fit_transform(df[num_features])

# Process textual features
text_features = ['title', 'category', 'cuisine', 'ingredients']
for feature in text_features:
    df[feature] = df[feature].fillna('').astype(str)
df['text_combined'] = df[text_features].apply(lambda row: ' '.join(row), axis=1)
vectorizer = TfidfVectorizer()
text_matrix = vectorizer.fit_transform(df['text_combined'])

# Combine numerical and textual features
num_matrix_sparse = 0.01 * num_matrix
num_matrix_sparse = csr_matrix(num_matrix_sparse)
final_matrix = hstack([text_matrix, num_matrix_sparse])

# 1. Vector Space Method using KNN
knn = NearestNeighbors(n_neighbors=11, metric='cosine')
knn.fit(final_matrix)

def vec_space_method(recipe_title):
    """Finds similar recipes using KNN."""
    indices = pd.Series(df.index, index=df['title'].str.lower()).drop_duplicates()
    recipe_index = indices.get(recipe_title.lower())
    if recipe_index is None:
        print(f"Recipe '{recipe_title}' not found.")
        return []
    distances, indices = knn.kneighbors(final_matrix[recipe_index])
    return df['title'].iloc[indices[0]].tolist()[1:]  # Exclude the input recipe itself

# 2. KNN Similarity method (same as vec_space_method in this implementation)
def knn_similarity(recipe_title):
    return vec_space_method(recipe_title)

# 3. Evaluation
test_users = {
    'User 1': 'Chicken tikka masala',
    'User 2': 'Albanian baked lamb with rice',
    'User 3': 'Baked salmon with chorizo rice',
    'User 4': 'Almond lentil stew'
}

def evaluate_recommendation_systems(test_users, vec_space_method, knn_similarity):
    """Evaluates the recommendation systems."""
    coverage_vec = 0
    coverage_knn = 0
    personalization_vec = 0
    personalization_knn = 0

    for user, recipe in test_users.items():
        vec_recommendations = vec_space_method(recipe)
        knn_recommendations = knn_similarity(recipe)

        if vec_recommendations:
            coverage_vec += 1
            personalization_vec += len(set(vec_recommendations)) / len(vec_recommendations)

        if knn_recommendations:
            coverage_knn += 1
            personalization_knn += len(set(knn_recommendations)) / len(knn_recommendations)

    coverage_vec /= len(test_users)
    coverage_knn /= len(test_users)
    personalization_vec /= len(test_users)
    personalization_knn /= len(test_users)

    print(f"Vector Space Method Coverage: {coverage_vec:.2f}")
    print(f"Vector Space Method Personalization: {personalization_vec:.2f}")
    print(f"KNN Similarity Coverage: {coverage_knn:.2f}")
    print(f"KNN Similarity Personalization: {personalization_knn:.2f}")

evaluate_recommendation_systems(test_users, vec_space_method, knn_similarity)

# 4. Classification Model
df['tasty'] = (df['rating_val'].astype(float) >= 4.2).astype(int)
df_filtered = df.dropna(subset=['rating_val'])

X_train, X_test, y_train, y_test = train_test_split(final_matrix[df_filtered.index], df_filtered['tasty'], test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Classification Model Accuracy: {accuracy:.2f}")

# Colabs

Machine Learning Crash Course uses Colaboratories (Colabs) for all programming exercises. Colab is Google's implementation of [Jupyter Notebook](https://jupyter.org/). For more information about Colabs and how to use them, go to [Welcome to Colaboratory](https://research.google.com/colaboratory).

# Pandas DataFrame UltraQuick Tutorial

This Colab introduces [**DataFrames**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), which are the central data structure in the pandas API. This Colab is not a comprehensive DataFrames tutorial.  Rather, this Colab provides a very quick introduction to the parts of DataFrames required to do the other Colab exercises in Machine Learning Crash Course.

A DataFrame is similar to an in-memory spreadsheet. Like a spreadsheet:

  * A DataFrame stores data in cells.
  * A DataFrame has named columns (usually) and numbered rows.

## Import NumPy and pandas modules

Run the following code cell to import the NumPy and pandas modules.

In [ ]:
import numpy as np
import pandas as pd

## Creating a DataFrame

The following code cell creates a simple DataFrame containing 10 cells organized as follows:

  * 5 rows
  * 2 columns, one named `temperature` and the other named `activity`

The following code cell instantiates a `pd.DataFrame` class to generate a DataFrame. The class takes two arguments:

  * The first argument provides the data to populate the 10 cells. The code cell calls `np.array` to generate the 5x2 NumPy array.
  * The second argument identifies the names of the two columns.

**Note**: Do not redefine variables in the following code cell. Subsequent code cells use these variables.

In [ ]:
# Create and populate a 5x2 NumPy array.
my_data = np.array([[0, 3], [10, 7], [20, 9], [30, 14], [40, 15]])

# Create a Python list that holds the names of the two columns.
my_column_names = ['temperature', 'activity']

# Create a DataFrame.
my_dataframe = pd.DataFrame(data=my_data, columns=my_column_names)

# Print the entire DataFrame
print(my_dataframe)

## Adding a new column to a DataFrame

You may add a new column to an existing pandas DataFrame just by assigning values to a new column name. For example, the following code creates a third column named `adjusted` in `my_dataframe`:

In [ ]:
# Create a new column named adjusted.
my_dataframe["adjusted"] = my_dataframe["activity"] + 2

# Print the entire DataFrame
print(my_dataframe)

## Specifying a subset of a DataFrame

Pandas provide multiples ways to isolate specific rows, columns, slices or cells in a DataFrame.

In [ ]:
print("Rows #0, #1, and #2:")
print(my_dataframe.head(3), '\n')

print("Row #2:")
print(my_dataframe.iloc[[2]], '\n')

print("Rows #1, #2, and #3:")
print(my_dataframe[1:4], '\n')

print("Column 'temperature':")
print(my_dataframe['temperature'])

## Task 1: Create a DataFrame

Do the following:

  1. Create an 3x4 (3 rows x 4 columns) pandas DataFrame in which the columns are named `Eleanor`,  `Chidi`, `Tahani`, and `Jason`.  Populate each of the 12 cells in the DataFrame with a random integer between 0 and 100, inclusive.

  2. Output the following:

     * the entire DataFrame
     * the value in the cell of row #1 of the `Eleanor` column

  3. Create a fifth column named `Janet`, which is populated with the row-by-row sums of `Tahani` and `Jason`.

To complete this task, it helps to know the NumPy basics covered in the NumPy UltraQuick Tutorial.


In [ ]:
# Write your code here.

In [ ]:
#@title Double-click for a solution to Task 1.

# Create a Python list that holds the names of the four columns.
my_column_names = ['Eleanor', 'Chidi', 'Tahani', 'Jason']

# Create a 3x4 numpy array, each cell populated with a random integer.
my_data = np.random.randint(low=0, high=101, size=(3, 4))

# Create a DataFrame.
df = pd.DataFrame(data=my_data, columns=my_column_names)

# Print the entire DataFrame
print(df)

# Print the value in row #1 of the Eleanor column.
print("\nSecond row of the Eleanor column: %d\n" % df['Eleanor'][1])

# Create a column named Janet whose contents are the sum
# of two other columns.
df['Janet'] = df['Tahani'] + df['Jason']

# Print the enhanced DataFrame
print(df)

## Copying a DataFrame (optional)

Pandas provides two different ways to duplicate a DataFrame:

* **Referencing.** If you assign a DataFrame to a new variable, any change to the DataFrame or to the new variable will be reflected in the other.
* **Copying.** If you call the `pd.DataFrame.copy` method, you create a true independent copy.  Changes to the original DataFrame or to the copy will not be reflected in the other.

The difference is subtle, but important.

In [ ]:
# Create a reference by assigning my_dataframe to a new variable.
print("Experiment with a reference:")
reference_to_df = df

# Print the starting value of a particular cell.
print("  Starting value of df: %d" % df['Jason'][1])
print("  Starting value of reference_to_df: %d\n" % reference_to_df['Jason'][1])

# Modify a cell in df.
df.at[1, 'Jason'] = df['Jason'][1] + 5
print("  Updated df: %d" % df['Jason'][1])
print("  Updated reference_to_df: %d\n\n" % reference_to_df['Jason'][1])

# Create a true copy of my_dataframe
print("Experiment with a true copy:")
copy_of_my_dataframe = my_dataframe.copy()

# Print the starting value of a particular cell.
print("  Starting value of my_dataframe: %d" % my_dataframe['activity'][1])
print("  Starting value of copy_of_my_dataframe: %d\n" % copy_of_my_dataframe['activity'][1])

# Modify a cell in df.
my_dataframe.at[1, 'activity'] = my_dataframe['activity'][1] + 3
print("  Updated my_dataframe: %d" % my_dataframe['activity'][1])
print("  copy_of_my_dataframe does not get updated: %d" % copy_of_my_dataframe['activity'][1])